In [1]:
from sklearn.linear_model import LinearRegression
import csv
import numpy as np

In [2]:
rgb = []
cmyk = []
with open('prismacolor.csv') as f:
    reader = csv.reader(f)
    for row in reader:
        if row[0] == 'Code':
            continue
        cmyk.append([int(row[2]), int(row[3]), int(row[4]), int(row[5])])
        rgb.append([int(row[6]), int(row[7]), int(row[8])])

In [3]:
cmyk = np.array(cmyk, 'float32')
rgb = np.array(rgb, 'float32')

In [4]:
def make_params(a):
    n, p = a.shape
    a2 = np.empty((n, p + p * p), dtype='float32')
    a2[:, 0:p] = a
    for i in range(p):
        s = p + i * p
        e = p + (i + 1) * p
        a2[:, s:e] = a * a[:, 0].reshape((n, 1))
    
    return a2
    
cmyk_ip = make_params(cmyk)
rgb_ip = make_params(rgb)

In [5]:
rgb2cmyk = LinearRegression()
rgb2cmyk.fit(rgb_ip, cmyk)

cmyk2rgb = LinearRegression()
cmyk2rgb.fit(cmyk_ip, rgb)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [6]:
rgb2cmyk.coef_, cmyk2rgb.coef_

(array([[-4.1336375e-01,  8.7289475e-02,  7.3187403e-02,  1.0815263e-04,
         -1.0160729e-04, -9.4035640e-05,  1.4278665e-04, -1.0160729e-04,
         -1.0502897e-04,  1.5603006e-04, -1.0160729e-04, -1.0226108e-04],
        [ 1.5752211e-01, -3.7863362e-01,  8.7546282e-02, -4.1827559e-05,
         -3.3967197e-05, -1.3422221e-04, -6.1482191e-05, -3.3952296e-05,
         -1.2561679e-04, -6.7383051e-05, -3.3952296e-05, -1.2778491e-04],
        [ 1.1796001e-01,  6.8308659e-02, -3.6564285e-01, -4.5396388e-05,
         -7.7046454e-05, -4.4828281e-05, -5.1490963e-05, -7.6994300e-05,
         -5.1742420e-05, -5.6492165e-05, -7.6994300e-05, -4.9985014e-05],
        [-1.9851278e-01, -1.7868930e-01, -1.9205138e-01, -9.6373260e-05,
          2.4206191e-04,  2.8530136e-04, -8.1904233e-05,  2.4209172e-04,
          2.7433410e-04, -7.7478588e-05,  2.4209172e-04,  2.7709827e-04]],
       dtype=float32),
 array([[-2.4635866e+00, -6.2505625e-02,  2.5828907e-02, -2.6142755e+00,
         -4.1303225e-04

In [7]:
cmyk[0], rgb[0]

(array([53.,  6., 84.,  0.], dtype=float32),
 array([120., 240.,  41.], dtype=float32))

In [8]:
cmyk2rgb.predict(cmyk_ip[0:1, :]), rgb2cmyk.predict(rgb_ip[0:1, :])

(array([[120.5126  , 226.35068 ,  60.237167]], dtype=float32),
 array([[48.390144  , -0.41435242, 84.66801   ,  8.982529  ]],
       dtype=float32))

In [9]:
cmyk_scale = 100
def rgb_to_cmyk(r,g,b):
    if (r == 0) and (g == 0) and (b == 0):
        # black
        return 0, 0, 0, cmyk_scale

    # rgb [0,255] -> cmy [0,1]
    c = 1 - r / 255.
    m = 1 - g / 255.
    y = 1 - b / 255.

    # extract out k [0,1]
    min_cmy = min(c, m, y)
    c = (c - min_cmy) / (1 - min_cmy)
    m = (m - min_cmy) / (1 - min_cmy)
    y = (y - min_cmy) / (1 - min_cmy)
    k = min_cmy

    # rescale to the range [0,cmyk_scale]
    return c*cmyk_scale, m*cmyk_scale, y*cmyk_scale, k*cmyk_scale

rgb_to_cmyk(rgb[0, 0], rgb[0, 1], rgb[0, 2])

(50.0, 0.0, 82.91666666666667, 5.882352941176472)